In [1]:
# Code adapted from Machine Learning Engineering (Cornell Tech 2025)
import torch
import numpy as np
import random

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [5]:
from google.colab import drive
import sys
import os

# --- 1. Mount Drive ---
drive.mount('/content/drive')

# --- 2. Define Paths ---
# Path to the source code (loaders.py) - REMAINS ON DRIVE
DRIVE_CODE_PATH = '/content/drive/MyDrive/GoogleColab/dataforsptransformer/'

# Path to the zipped data file on Drive
ZIP_SOURCE_PATH = os.path.join(DRIVE_CODE_PATH, 'crc_markers')

# Local disk folder where the FAST images will be unzipped
FAST_DATA_PATH = '/content/fast_data/'

# --- 3. Unzip Data (Performance Fix) ---
print(f"🚀 Unzipping data from Drive to fast local disk: {FAST_DATA_PATH}")
!mkdir -p "$FAST_DATA_PATH"
# The -q flag silences the output. -d sets the destination directory.
!unzip -q "$ZIP_SOURCE_PATH" -d "$FAST_DATA_PATH"

print("✅ Data transfer complete. Starting new batch load test.")


# --- 4. Set Final Variables ---
# PROJECT_DIR for the rest of your notebook now points to the FAST images
PROJECT_DIR = FAST_DATA_PATH

# Add the Drive path for Python to find 'loaders.py' and other modules
if DRIVE_CODE_PATH not in sys.path:
    sys.path.append(DRIVE_CODE_PATH)
    print(f"✅ Added {DRIVE_CODE_PATH} to Python system path.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🚀 Unzipping data from Drive to fast local disk: /content/fast_data/
unzip:  cannot find or open /content/drive/MyDrive/GoogleColab/dataforsptransformer/crc_markers, /content/drive/MyDrive/GoogleColab/dataforsptransformer/crc_markers.zip or /content/drive/MyDrive/GoogleColab/dataforsptransformer/crc_markers.ZIP.
✅ Data transfer complete. Starting new batch load test.
